In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import pandas as pd
import numpy as np
import json
import xarray as xr


In [47]:
metadata_file = '../../data/ambient_network.json'

with open(metadata_file) as f:
    ambient_network = json.load(f)
    
#ambient_network

In [65]:
wrtdsk_load = pd.read_csv('../../data/illinois_ambient_annual_loads_wrtdsk.csv', dtype={'site':str})
wrtdsk_load['year'] = pd.to_datetime(wrtdsk_load['year'], format='%Y').values
wrtdsk_load = wrtdsk_load.set_index(['site','year'])

wrtdsk_load.head()

total suspended solids [kg]  \
site     year                                      
03336645 1979-01-01                 4.441585e+08   
         1980-01-01                 2.823931e+08   
         1981-01-01                 1.886789e+08   
         1982-01-01                 5.648412e+08   
         1983-01-01                 9.445388e+07   

                     volatile suspended solids [kg]  total nitrogen [kg]  \
site     year                                                              
03336645 1979-01-01                             NaN           5247228.68   
         1980-01-01                             NaN           2893754.65   
         1981-01-01                             NaN           4858619.78   
         1982-01-01                             NaN           6185522.08   
         1983-01-01                             NaN           3856953.07   

                     ammonia nitrogen [kg]  organic nitrogen [kg]  \
site     year                                                       
03336645 1979-01-01                    NaN             2389157.45   
         1980-01-01                    NaN             1028470.86   
         1981-01-01                    NaN              977276.77   
         1982-01-01                    NaN             2942128.68   
         1983-01-01                    NaN              730680.37   

                     nitrate nitrogen [kg]  total phosphorus [kg]  \
site     year                                                       
03336645 1979-01-01             3804656.32              210173.74   
         1980-01-01             2028750.03               88583.75   
         1981-01-01             3475216.04                    NaN   
         1982-01-01             4068385.10                    NaN   
         1983-01-01             3071834.27                    NaN   

                     dissolved phosphorus [kg]  particulate phosphorus [kg]  \
site     year                                                                 
03336645 1979-01-01                        NaN                          NaN   
         1980-01-01                        NaN                          NaN   
         1981-01-01                   38629.77                          NaN   
         1982-01-01                   85213.76                          NaN   
         1983-01-01                   31662.84                          NaN   

                     sulfate [kg]  ...  organic nitrogen [count]  \
site     year                      ...                             
03336645 1979-01-01           NaN  ...                      12.0   
         1980-01-01           NaN  ...                      13.0   
         1981-01-01           NaN  ...                      12.0   
         1982-01-01           NaN  ...                       8.0   
         1983-01-01           NaN  ...                       8.0   

                     nitrate nitrogen [count]  total phosphorus [count]  \
site     year                                                             
03336645 1979-01-01                      12.0                      12.0   
         1980-01-01                      13.0                       5.0   
         1981-01-01                       9.0                       0.0   
         1982-01-01                       8.0                       0.0   
         1983-01-01                       8.0                       0.0   

                     dissolved phosphorus [count]  \
site     year                                       
03336645 1979-01-01                           0.0   
         1980-01-01                           0.0   
         1981-01-01                           9.0   
         1982-01-01                           8.0   
         1983-01-01                           8.0   

                     particulate phosphorus [count]  sulfate [count]  \
site     year                                                          
03336645 1979-01-01                             0.0              0.0   
         1

In [78]:
wrtdsk_load = wrtdsk_load.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,-2,-1]]

In [88]:
wrtdsk_load = wrtdsk_load.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,-2,-1]]

column_names = [name[:name.find('[') -1] for name in wrtdsk_load.columns]
unit_names = [name[name.find('[')+1:name.find(']')] for name in wrtdsk_load.columns]

pint_dictionary = {column:unit for column, unit in zip(column_names, unit_names)}
pint_dictionary

{'total suspended solids': 'kg',
 'volatile suspended solids': 'kg',
 'total nitrogen': 'kg',
 'ammonia nitrogen': 'kg',
 'organic nitrogen': 'kg',
 'nitrate nitrogen': 'kg',
 'total phosphorus': 'kg',
 'dissolved phosphorus': 'kg',
 'particulate phosphorus': 'kg',
 'sulfate': 'kg',
 'suspended sediment': 'kg',
 'chloride': 'kg',
 'flow': 'days',
 'mean flow': 'm^3/sec'}

In [89]:
wrtdsk_load.columns = column_names
wrtdsk_ds = wrtdsk_load.to_xarray()
#wrtdsk_ds = wrtdsk_ds[load_cols].rename(name_dict=name_dict)


wrtdsk_ds = wrtdsk_ds.pint.quantify(pint_dictionary)

d = wrtdsk_ds.pint.dequantify()
d.to_netcdf('../../data/illinois_ambient_annual_loads_wrtdsk.nc')
#

In [ ]:
#supergages

In [17]:
#!gdown https://www.sciencebase.gov/catalog/file/get/61f97181d34e622189c510ad?f=__disk__09%2F1c%2Fd6%2F091cd6dd4d7272d2f484c8bd148e174adef2553e
import pandas as pd

cm_df = pd.read_csv('../../data/Illinois_nutrient_and_sediment_concentrations_wy2016-2021.csv', dtype={'USGS_site':str}, parse_dates=['datetime']).rename({'USGS_site':'site'}, axis=1)
cm_df = cm_df.set_index(['site','datetime'])

In [18]:
# convert concentrations to loads
CF_to_L = 28.3168
MG_to_KG = 1e-6
sec_to_day = 86400.0
conversion = CF_to_L * MG_to_KG * sec_to_day
cm_df['nitrate nitrogen'] = cm_df['q'] * cm_df['no3_mg_per_l'] * conversion
cm_df['total phosphorus'] = cm_df['q'] * cm_df['total_p_mg_per_l'] * conversion

In [20]:
import xarray as xr
import pint_xarray

load_cols = ['nitrate nitrogen','total phosphorus']
#name_dict = {'00630_kg':'nitrate', '00665_kg':'phosphorus'}
cm_ds = cm_df.to_xarray()
cm_ds = cm_ds[load_cols]
cm_ds['water_year'] = cm_ds.datetime.dt.year.where(cm_ds.datetime.dt.month < 10, cm_ds.datetime.dt.year + 1)

#cm_ds

In [21]:
#cm_annual = cm_ds.groupby('datetime.year').sum().pint.quantify({'nitrate': 'kg', 'phosphorus': 'kg'})
cm_annual = cm_ds.groupby('water_year').sum().pint.quantify({'nitrate nitrogen': 'kg', 'total phosphorus': 'kg'})
cm_annual['water_year'] = pd.to_datetime(cm_annual['water_year'], format='%Y').values
cm_annual = cm_annual.rename({'water_year':'year'})
#
d = cm_annual.pint.dequantify()
d.to_netcdf('../../data/illinois_supergage_annual_loads_continuous.nc')

# old

In [13]:
wrtdsk_load = pd.read_csv('../../data/Illinois_annual_wrtds-k_loads.csv', dtype={'site_no':str})

wrtdsk_load['year'] = pd.to_datetime(wrtdsk_load['year'], format='%Y').values
wrtdsk_load = wrtdsk_load.set_index(['site_no','year'])

wrtdsk_load.head()
#wrtdsk_load.index.dtypes

00530_kg  00535_kg    00600_kg  00610_kg    00625_kg  \
site_no  year                                                                   
03336645 1979-01-01  4.245040e+08       NaN  5211629.08       NaN  2073863.64   
         1980-01-01  2.530749e+08       NaN  2918988.52       NaN  1026367.54   
         1981-01-01  2.041528e+08       NaN  4654886.00       NaN   945191.74   
         1982-01-01  5.166362e+08       NaN  6150387.24       NaN  2975568.52   
         1983-01-01  9.635003e+07       NaN  3891292.98       NaN   723070.12   

                       00630_kg   00665_kg  00666_kg  00667_kg     00946_kg  \
site_no  year                                                                 
03336645 1979-01-01  3759277.98  205410.92       NaN       NaN  23269956.76   
         1980-01-01  2070057.70   92282.17       NaN       NaN  13971288.33   
         1981-01-01  3442636.84        NaN  35899.39       NaN  18745524.02   
         1982-01-01  4238406.61        NaN  89622.15       NaN  27195534.22   
         1983-01-01  3133585.88        NaN  32643.86       NaN  20467948.32   

                     ...  00610_n  00625_n  00630_n  00665_n  00666_n  \
site_no  year        ...                                                
03336645 1979-01-01  ...        0       12       12       12        0   
         1980-01-01  ...        0       13       13        5        0   
         1981-01-01  ...        0       12        9        0        9   
         1982-01-01  ...        0        8        8        0        8   
         1983-01-01  ...        0        8        8        0        8   

                     00667_n  00946_n  80154_n  99220_n  flow_days  
site_no  year                                                       
03336645 1979-01-01        0       12        0        0        365  
         1980-01-01        0       13        0       13        366  
         1981-01-01        0       12        0       12        365  
         1982-01-01        0        8        0        8        365  
         1983-01-01        0        8        0        8        365  

[5 rows x 37 columns]

In [38]:
import xarray as xr
import pint_xarray


load_cols = ['00630_kg','00665_kg']
name_dict = {'00630_kg':'nitrate', '00665_kg':'phosphorus'}
wrtdsk_ds = wrtdsk_load.to_xarray()
wrtdsk_ds = wrtdsk_ds[load_cols].rename(name_dict=name_dict)


wrtdsk_ds = wrtdsk_ds.pint.quantify({'nitrate': 'kg', 'phosphorus': 'gse'})

d = wrtdsk_ds.pint.dequantify()
d.to_netcdf('../../data/ambient_wrtdsk_annual_loads.netcdf')
#wrtdsk_ds.pint.dequantify()
#wrtdsk_ds.pint.dequantify()


ValueError: Cannot parse units:
    invalid units for variable 'phosphorus': gse (parameter) (reason: 'gse' is not defined in the unit registry)

<function pint.get_application_registry()>

In [6]:
# format continuous data

In [7]:
#!gdown https://www.sciencebase.gov/catalog/file/get/61f97181d34e622189c510ad?f=__disk__09%2F1c%2Fd6%2F091cd6dd4d7272d2f484c8bd148e174adef2553e
import pandas as pd

cm_df = pd.read_csv('../../data/Illinois_nutrient_and_sediment_concentrations_wy2016-2021.csv', dtype={'USGS_site':str}, parse_dates=['datetime']).rename({'USGS_site':'site_no'}, axis=1)
cm_df = cm_df.set_index(['site_no','datetime'])

In [8]:
# convert concentrations to loads
CF_to_L = 28.3168
MG_to_KG = 1e-6
sec_to_day = 86400.0
conversion = CF_to_L * MG_to_KG * sec_to_day
cm_df['nitrate'] = cm_df['q'] * cm_df['no3_mg_per_l'] * conversion
cm_df['phosphorus'] = cm_df['q'] * cm_df['total_p_mg_per_l'] * conversion

In [9]:
import xarray as xr
import pint_xarray

load_cols = ['nitrate','phosphorus']
#name_dict = {'00630_kg':'nitrate', '00665_kg':'phosphorus'}
cm_ds = cm_df.to_xarray()
cm_ds = cm_ds[load_cols]
cm_ds['water_year'] = cm_ds.datetime.dt.year.where(cm_ds.datetime.dt.month < 10, cm_ds.datetime.dt.year + 1)

#cm_ds

In [10]:
#def water_year(datetime):
#    return datetime.dt.month

In [ ]:
#cm_annual = cm_ds.groupby('datetime.year').sum().pint.quantify({'nitrate': 'kg', 'phosphorus': 'kg'})
cm_annual = cm_ds.groupby('water_year').sum().pint.quantify({'nitrate': 'kg', 'phosphorus': 'kg'})
cm_annual['water_year'] = pd.to_datetime(cm_annual['water_year'], format='%Y').values
cm_annual = cm_annual.rename({'water_year':'year'})
#
d = cm_annual.pint.dequantify()
d.to_netcdf('../../data/supergage_annual_loads.netcdf')


In [ ]:
# upload achive to aws

import xarray as xr
import s3fs
fs_s3 = s3fs.S3FileSystem(anon=True)
s3path = 'era5-pds/2008/01/data/air_temperature_at_2_metres.nc'
remote_file_obj = fs_s3.open(s3path, mode='rb')
ds = xr.open_dataset(remote_file_obj, engine='h5netcdf')

# scratch

In [ ]:
cm_ds.sel(site_no='03339000').mean()

In [ ]:
test = xr.load_dataset('../../data/ambient_wrtdsk_annual_loads.netcdf')
test.sel(site_no='03339000').mean()

In [ ]:
cm_df.index.levels[1].year

In [ ]:
#wrtdsk_ds['00630_kg']

In [ ]:
#wrtdsk_ds['00630_kg']

In [ ]:
site_load = compute_network_loads(wrtdsk_ds, ambient_network)

In [ ]:
def location_label(site):
    return site['river'] + ' ' + site['location']

def compute_river_load(ds, site, nested=True):
    site_load = ds.sel(site_no=site['gage_id'])
    site_load = site_load.drop_vars('site_no')
    site_load = site_load * site['scale_factor']


    upstream_gage = site.get('upstream_gage')
    
    if upstream_gage and nested:
        upstream_load = ds.sel(site_no=upstream_gage).drop_vars('site_no')
        site_load = site_load - upstream_load
        
    site_load = site_load.assign_coords(coords={'river':location_label(site)})
    
    return site_load


def compute_network_loads(ds, network, nested=True):
    rivers = []
    
    for site in network:
        rivers.append(compute_river_load(ds, site, nested))
        
    return xr.concat(rivers, dim='river')
    #return rivers

def network_labels(network):
    labels = []
    for site in network:
        nested = site.get('nested')

        if nested is None or nested==False:
            #print(nested)
            labels.append(location_label(site))
            
    return labels
#def compute_network_total(ds, network):
#    pass

def kg_to_lbs(x):
    return x * 2.20462

def kg_to_kt(x):
    return x * 1.0e-6

In [ ]:
rivers = compute_network_loads(wrtdsk_ds, ambient_network)

In [ ]:
network_labels(ambient_network)

In [ ]:


ds = rivers.sel(river = network_labels(ambient_network)).sel(year=slice(1980,2021)).apply(kg_to_kt) # .sum(dim='river')

In [ ]:
baseline = ds.sel(year=slice(1980,1996))

last5 = ds.sel(year=slice(2016,2021))

In [ ]:
baseline.mean(dim='year')

In [ ]:
last5.mean(dim='year')

In [ ]:
rivers.sel(river='Rock at Joslin').plot.scatter(x='year',y='nitrate')

In [ ]:
rivers.sel(river='Illinois at Valley City').plot.scatter(x='year',y='nitrate')

In [ ]:


rivers.sel(year=slice(1980,2021)).sel(river='Rock at Rockton')#river